In [ ]:
!pip install pyomo
!wget https://people.montefiore.uliege.be/cornelusse/utils/ampl.linux-intel64.zip
!unzip /content/ampl.linux-intel64.zip
!chmod +x /content/ampl.linux-intel64/ipopt # To make sure it can be executed
!chmod +x /content/ampl.linux-intel64/cbc # To make sure it can be executed

In [ ]:
!pip install pandapower --upgrade

In [ ]:
!pip install networkx

In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed

In [ ]:
import gurobipy as gp
import time
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass
import pyomo.environ as pyo
import math
import pandapower as pp


# Create a new graph
G = nx.DiGraph()

In [ ]:
import pandapower.networks as pn
import pandapower as pp

def modified_case_14():
    net = pn.case14()

    # Find line indices for (9,10) and (13,14)
    line_idx = net.line[
        ((net.line.from_bus == 8) & (net.line.to_bus == 9)) |   # buses are 0-based in pandapower
        ((net.line.from_bus == 12) & (net.line.to_bus == 13))
    ].index

    # Find trafo index for (7,9)
    trafo_idx = net.trafo[
        ((net.trafo.hv_bus == 6) & (net.trafo.lv_bus == 8))
    ].index

    # Drop them
    pp.drop_lines(net, line_idx)
    pp.drop_trafos(net, trafo_idx)

    return net

net = modified_case_14()


In [ ]:
import pandapower.plotting.plotly as ppply

# Create an interactive plot with voltage information on hover
ppply.pf_res_plotly(net, filename="interactive_network_plot.html", cmap='viridis')

In [ ]:
import pandapower.plotting.plotly as ppply

# Create an interactive plot with voltage information on hover
ppply.pf_res_plotly(pn.case14(), filename="interactive_network_plot.html", cmap='viridis')

In [ ]:
s_base = 100 #MVA

In [ ]:
def calculate_current_bounds(v_base):
    """
    Calculates the upper and lower bounds for current in per-unit
    based on the line type's maximum current capacity.

    Args:
        line_type (str): The type of the transmission line.
        i_base_kA (float): The base current in kA for the system.

    Returns:
        tuple: A tuple containing (I_min_pu, I_max_pu).
               Returns (None, None) if the line type is not found.
    """
    # Define a dictionary mapping line types to their maximum current capacity in kA
    # NOTE: This is a placeholder. You should populate this dictionary
    # with the actual maximum current capacities for your line types.
    i_base_kA = s_base / (math.sqrt(3) * v_base) # kA


    max_current_kA = {
        135: 0.96, # Example value for 110 kV lines
        14: 0.535, # Example value for 20 kV lines
        0.208: 0.35 # Example value for 0.4 kV lines
    }

    if v_base in max_current_kA:
        I_max_kA = max_current_kA[v_base]

        # Calculate the upper bound for current in per-unit
        I_max_pu = I_max_kA / i_base_kA

        # The lower bound for current is the negative of the upper bound
        I_min_pu = -I_max_pu

        return I_min_pu, I_max_pu

    elif v_base not in max_current_kA:
        return -1000, 1000
    else:
        return None, None


#I_min_line1, I_max_line1, i_base_kA = calculate_current_bounds(line1_type, i_base_high)

# line4_type = 'NA2XS2Y 1x185 RM/25 12/20 kV'
I_min_line4, I_max_line4 = calculate_current_bounds(14)
print([{I_min_line4}, {I_max_line4}])

In [ ]:
def create_graph(net) -> nx.Graph:
    import json
    # -------------------------
    # 1. Conversion du réseau Pandapower en DiGraph
    # -------------------------
    G = nx.Graph()

    # 2. Récupération des positions à partir de la colonne 'geo'
    pos = {}
    for idx, row in net.bus.iterrows():
        geo_data = row['geo']
        if geo_data is None:
            raise ValueError(f"Le bus '{row['name']}' (index {idx}) n'a pas de coordonnées 'geo'.")
        geo_dict = json.loads(geo_data)
        coordinates = geo_dict["coordinates"]
        pos[idx] = tuple(coordinates)

    #Ajouter une puissance de base
    G.graph["s_base"] = 100.0 # MVA
    G.graph["v_base"] = 110.0 # kV

    # Ajouter les nœuds
    for idx, row in net.bus.iterrows():
        G.add_node(idx,
                   label=row["name"],
                   pos=pos[idx],
                   vn_kv=row["vn_kv"])

    # Ajouter les arêtes pour les lignes
    for _, row in net.line.iterrows():
        G.add_edge(row["from_bus"], row["to_bus"],
                   type="line",
                   name=row["name"],
                   length=row["length_km"],
                   std_type=row["std_type"],
                   x_ohm=row["x_ohm_per_km"],)

    # Ajouter les arêtes pour les transformateurs
    for _, row in net.trafo.iterrows():
        G.add_edge(row["hv_bus"], row["lv_bus"],
                   type="trafo", std_type = None,
                   name=row["name"])

    # Ajouter les générateurs et les charges comme attributs aux nœuds
    for _, row in net.gen.iterrows():
        G.nodes[row["bus"]]["type"] = "gen"
        G.nodes[row["bus"]]["gen_name"] = row["name"]
        G.nodes[row["bus"]]["gen_power"] = row["p_mw"]

    for _, row in net.load.iterrows():
        G.nodes[row["bus"]]["type"] = "load"
        G.nodes[row["bus"]]["load_name"] = row["name"]
        G.nodes[row["bus"]]["load_power"] = row["p_mw"]

    for _, row in net.ext_grid.iterrows():
        G.nodes[row["bus"]]["type"] = "ext_grid"
        G.nodes[row["bus"]]["grid_name"] = row["name"]

    # -------------------------
    # 2. Manually define power demand and add to nodes
    # -------------------------
    # Manually define the 'P' attribute for each node in the graph G
    # Example:
    co = 4
    power_demand = {
        0: 0.0/co,
        1: 50,
        2: -20,
        3: 0/co,
        4: 0/co,
        5: -1e-1/co,
        6: 0.1/co,
        7: -0.2/co,
        8: -29.5*1e-2 /co,
        9: -9.0*1e-1 /co,
        10: -3.5*1e-1 /co,
        11: -6.1*1e-1 /co,
        12: -13.5*1e-2 /co,
        13: -14.9*1e-2 /co
    }

    for n in G.nodes:
        G.nodes[n]["P"] = power_demand.get(n, 0.0) # Use get with a default value of 0.0
        G.nodes[n]["P_pu"] = G.nodes[n]["P"]/s_base
    print(net.line.columns)

    # Calculate the susceptance and max_current of each line in Siemens
    for u, v, data in G.edges(data=True):
        if data.get('type') == 'trafo':
             # Assign a high max_current for transformers
             data['max_current_pu'] = 1000 # Use a large value
             print(f"Transformer {u}->{v}: max_current_pu = {data['max_current_pu']} pu (high value)")
        else:
            data['b_pu'] = (G.nodes[u]["vn_kv"]**2 / (data['x_ohm'] * G.graph["s_base"]))  # Calcule et stocke B_ij per unit
            print(f"Ligne {u}->{v}: b_pu = {G[u][v]['b_pu']} pu")

            # Calculate max_current for lines based on line type and node voltage
            line_type = data.get('std_type')
            v_node_kv = G.nodes[u].get("vn_kv") # Get voltage of the 'from' node

            if v_node_kv is not None:
                I_min_pu, I_max_pu = calculate_current_bounds(v_node_kv)
                if I_max_pu is not None:
                    data['max_current_pu'] = I_max_pu
                    print(f"Line {u}->v: max_current_pu = {data['max_current_pu']} pu")
                else:
                    print(f"Warning: Could not calculate max_current for line {u}->{v} (unknown line type or voltage).")
                    data['max_current_pu'] = None # Or a default value if appropriate
            else:
                 print(f"Warning: Missing line type or node voltage for line {u}->{v}. Could not calculate max_current.")
                 data['max_current_pu'] = None # Or a default value if appropriate


    B_base = G.graph["s_base"]/(G.graph["v_base"]**2)
    print("B_base: ", B_base)

    # -------------------------
    # Donner accès à G
    # -------------------------
    node_attrs = {n: G.nodes[n] for n in G.nodes}
    return G # Return G and the plot_network function

In [ ]:
G = create_graph(net)
print(f"Value of G.nodes[11]['P'] after updating G: {G.nodes[11]['P']}")

In [ ]:
# 3. Préparer les couleurs des nœuds en fonction de P_net
# ================================
node_colors = []
for n, data in G.nodes(data=True):
    if G.nodes[n]["P"] > 0:
        node_colors.append("green")   # producteur
    elif G.nodes[n]["P"] < 0:
        node_colors.append("red")     # consommateur
    else:
        node_colors.append("gray")    # neutre

# 4. Préparer les labels : Nom + P_net
# ================================
labels = {n: f"{data['label']}\nP={round(G.nodes[n]["P_pu"],7)}"
          for n, data in G.nodes(data=True)}

def plot_network():
      pos = nx.get_node_attributes(G, 'pos')

      plt.figure(figsize=(12, 8))
      nx.draw(
          G, pos,
          with_labels=True, labels=labels,
          node_size=1200, node_color=node_colors,
          edgecolors="black", font_size=8,
          alpha=0.85
          )

  # Labels des arêtes (type ligne ou trafo)
      edge_labels = nx.get_edge_attributes(G, 'type')
      nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)

      plt.title("Réseau électrique avec puissances (P_net)")
      plt.axis("equal")
      plt.show()
plot_network()

In [ ]:
def plot_network(G):
      pos = nx.get_node_attributes(G, 'pos')

      plt.figure(figsize=(12, 8))

      # Create labels with node index and power demand (P)
      labels = {n: f"{n}\nP={data.get('P', 0):.2f}" for n, data in G.nodes(data=True)}


      nx.draw(
          G, pos,
          with_labels=True, labels=labels, # Use the new labels
          node_size=1200,
          edgecolors="black", font_size=8,
          alpha=0.85
          )

  # Labels des arêtes (type ligne ou trafo)
      edge_labels = nx.get_edge_attributes(G, 'type')
      nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)

      plt.title("Réseau électrique avec puissances (P_net)")
      plt.axis("equal")
      plt.show()

plot_network(G)

In [ ]:
#Constant definition
V_min = 0.9
V_max = 1.1


P_min = -0.3
P_max = 0.3

theta_min = -1
theta_max = 1


alpha = 1


In [ ]:
m = pyo.ConcreteModel()
# Define base values for the per-unit system
s_base = 100  # MVA
v_base_high = 110 # kV (From the create_network function)

m.Nodes = pyo.Set(initialize= [b for b in G.nodes])
m.Lines = pyo.Set(initialize= [l for l in G.edges])
m.i = pyo.Set(initialize=[0, 1]) # Initialize m.i with two generic elements
m.j = pyo.Set(initialize= [0, 1])
#m.children = pyo.Set(initialize= [1, 2])
m.parents = pyo.Set(initialize = [0])

# Get the line indices from the model
line_indices = list(m.Lines)

# You can print the indices
print("Indices of all lines:")
for line in line_indices:
    print(line)

m.P = pyo.Param(m.Nodes, initialize = {n: - G.nodes[n].get('P_pu') for n in G.nodes}, domain = pyo.Reals, mutable=True) # Make P mutable

m.I_min = pyo.Param(m.Lines, initialize = {(i,j): calculate_current_bounds(G.nodes[i]["vn_kv"])[0] for (i,j) in m.Lines}, domain = pyo.Reals)
m.I_max = pyo.Param(m.Lines, initialize = {(i,j): calculate_current_bounds( G.nodes[i]["vn_kv"])[1] for (i,j) in m.Lines}, domain = pyo.Reals)

print(m.I_min[(0,1)])
print(m.I_max[(0,1)])

# Variables principales
m.F = pyo.Var(m.Lines, m.i, m.j, domain=pyo.Reals)     # active power flow through lines
m.I = pyo.Var(m.Lines,  m.i, m.j, domain=pyo.Reals)  # current flowing through lines
m.theta = pyo.Var(m.Nodes,  m.i, m.j, domain=pyo.Reals)  # phase angle of the voltage
m.V = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.NonNegativeReals)  # voltage magnitude at each node
m.E = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.Reals) #net power injection/consumption
m.P_plus = pyo.Var(m.parents, m.i, m.j, domain=pyo.Reals) #power entering the operational graph
#m.P_minus = pyo.Var(m.children, m.i, m.j, domain=pyo.Reals) #power leaving the operational graph

#m.P_C_set = pyo.Var(m.children, m.i, domain=pyo.Reals) #vertices of the power envelope at each child node

#m.P_C_min = pyo.Var(m.children, m.i, m.j, domain=pyo.Reals)
#m.P_C_max = pyo.Var(m.children, m.i, m.j,  domain=pyo.Reals)

m.V_P = pyo.Param(m.j, initialize = {0: 0.9, 1: 1.1}, domain =pyo.NonNegativeReals)

m.O = pyo.Var(domain= pyo.NonNegativeReals)

for u in G.nodes():
  if G.nodes[u].get('P', 0.0) == 0:
    m.P[u] = 0
  else:
    G.nodes[u]['P_pu'] = G.nodes[u].get('P', 0.0) / s_base
    m.P[u] = - G.nodes[u]['P_pu']

"""# Convert P parameter to per-unit
for n in G.nodes:
    # Assuming P is in MW, convert to per-unit
    G.nodes[n]['P'] = G.nodes[n].get('P', 0.0) / s_base
    m.P[n] = G.nodes[n]['P']"""

In [ ]:
for n in G.nodes:
  print(m.P[n].value)


In [ ]:
# Auxiliary variable for the absolute value of E (already defined as per-unit)
m.z = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.NonNegativeReals)

m.curt = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.Reals)

def curtailment_def_rule(m,n, vert_pow, vert_volt):

  return m.curt[n, vert_pow, vert_volt] == m.P[n] - m.E[n, vert_pow, vert_volt]

m.curt_def = pyo.Constraint(m.Nodes, m.i, m.j, rule=curtailment_def_rule)
# Constraints to define the absolute value (already defined for per-unit E)
def abs_E_pos_rule(m, n, vert_pow, vert_volt):
    return m.z[n, vert_pow, vert_volt] >= m.curt[n, vert_pow, vert_volt]
m.abs_E_pos = pyo.Constraint(m.Nodes, m.i, m.j, rule=abs_E_pos_rule)

def abs_E_neg_rule(m, n, vert_pow, vert_volt):
    return m.z[n, vert_pow, vert_volt] >= -m.curt[n, vert_pow, vert_volt]
m.abs_E_neg = pyo.Constraint(m.Nodes, m.i, m.j, rule=abs_E_neg_rule)

def upper_bound_rule(m, vert_pow, vert_volt):
    # m.z is per-unit, m.O is per-unit
    return sum(m.z[n, vert_pow, vert_volt] for n in m.Nodes) <= m.O

m.upper_bound = pyo.Constraint(m.i, m.j, rule=upper_bound_rule)

# Current magnitude constraint (I_min, I_max are assumed per-unit)
def current_bounds_rule(m, i, j, vert_pow, vert_volt):
    # m.I is per-unit current
    return pyo.inequality(m.I_min[(i,j)], m.I[i,j, vert_pow, vert_volt], m.I_max[(i,j)])
m.CurrentBounds = pyo.Constraint(m.Lines, m.i, m.j, rule=current_bounds_rule)

def phase_constr_rule(m,n, vert_pow, vert_volt):
    return pyo.inequality(theta_min, m.theta[n, vert_pow, vert_volt], theta_max)

m.phaseConstr = pyo.Constraint(m.Nodes, m.i, m.j, rule=phase_constr_rule)

def dc_power_flow_rule(m, i, j, vert_pow, vert_volt):
    if G[i][j].get('b_pu') is None:
      return pyo.Constraint.Skip
    else:
      return m.F[i, j, vert_pow, vert_volt] == m.V_P[vert_volt]**2 *(G[i][j]['b_pu'] * (
        m.theta[i, vert_pow, vert_volt] - m.theta[j, vert_pow, vert_volt])
    )

m.DCFlow = pyo.Constraint(m.Lines, m.i, m.j, rule=dc_power_flow_rule)


def current_def_rule(m, n, i, j, vert_pow, vert_volt):
    # This constraint relates per-unit current, per-unit voltage, and per-unit power flow.
    # In per-unit, P_pu = V_pu * I_pu. This is correct.
    return np.sqrt(3)* m.I[i,j, vert_pow, vert_volt] * m.V_P[vert_volt] == m.F[i,j, vert_pow, vert_volt]

m.current_def = pyo.Constraint(m.Nodes, m.Lines, m.i, m.j, rule=current_def_rule)



def power_balance_rule(m, n, vert_pow, vert_volt):
    # Compute net flow into node n by summing over all lines (i,j) in m.Lines
    expr = sum(
        (m.F[i, j, vert_pow, vert_volt] if j == n else 0)
      - (m.F[i, j, vert_pow, vert_volt] if i == n else 0)
      for (i, j) in m.Lines
    )
    # If n is a parent node, subtract P_plus; otherwise use only E[n]
    if n in m.parents:
        return expr == m.E[n, vert_pow, vert_volt] - m.P_plus[n, vert_pow, vert_volt]
    else:
        return expr == m.E[n, vert_pow, vert_volt]

m.power_balance = pyo.Constraint(m.Nodes, m.i, m.j, rule=power_balance_rule)

def parent_power_constraint_rule(m, parent, vert_pow, vert_volt):
    # m.P_plus is per-unit power entering the operational graph
    return pyo.inequality(P_min, m.P_plus[parent, vert_pow, vert_volt], P_max)

m.parent_power_constraint = pyo.Constraint(m.parents, m.i, m.j, rule=parent_power_constraint_rule)

#Constant voltage assumption
def voltage_constr_rule(m, n, vert_pow, vert_volt):
    return m.V[n, vert_pow, vert_volt] == m.V_P[vert_volt]

m.voltageConstr = pyo.Constraint(m.Nodes, m.i, m.j, rule=voltage_constr_rule)




"""def test_rule(m):
  return m.O == 0
m.test = pyo.Constraint(rule=test_rule)"""


"""# Auxiliary variables for the L1 norm of P_C_set
m.abs_P_C_set = pyo.Var(m.children, domain=pyo.NonNegativeReals)

# Constraints to linearize the L1 norm of P_C_set
def abs_P_C_set_pos_rule(m, n):
    return m.abs_P_C_set[n] >= m.aux[n]
m.abs_P_C_set_pos = pyo.Constraint(m.children, rule=abs_P_C_set_pos_rule)

def abs_P_C_set_neg_rule(m, n):
    return m.abs_P_C_set[n] >= -m.aux[n]
m.abs_P_C_set_neg = pyo.Constraint(m.children, rule=abs_P_C_set_neg_rule)"""

In [ ]:
def objective_rule(m):
  return   - alpha*m.O

m.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

In [ ]:
# Résolution
# -------------------------
# Create an environment with your WLS license
params = {
    "WLSACCESSID": 'efa10779-150a-495c-ab62-6a20ff44c598',
    "WLSSECRET": 'fa980d94-a6fd-4d7b-aa68-b0e19ddd2e84',
    "LICENSEID": 2651433,
}
env = gp.Env(params=params)
solver = pyo.SolverFactory('gurobi', env= env)

# Solve the model
results = solver.solve(m, tee=True)

# Check the termination condition
if results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    print("The model is infeasible.")
elif results.solver.termination_condition == pyo.TerminationCondition.unbounded:
    print("The model is unbounded.")
elif results.solver.termination_condition == pyo.TerminationCondition.infeasibleOrUnbounded:
    print("The model is infeasible or unbounded.")
else:
    print("Solver terminated with condition:", results.solver.termination_condition)
    # Print the results (optional, but good practice)
    print(results)

In [ ]:
m.O.value

In [ ]:
m.theta[0,0,0].value

In [ ]:
# Print values of m.I
print("Values of m.I:")
for (u, v) in m.Lines:
    for vert_pow in m.i:
        for vert_volt in m.j:
            try:
                print(f"m.I[{u},{v},{vert_pow},{vert_volt}].value: {m.I[u, v, vert_pow, vert_volt].value}")
            except:
                print(f"m.I[{u},{v},{vert_pow},{vert_volt}].value: None")


# Print values of m.I_max
print("\nValues of m.I_max:")
for (u, v) in m.Lines:
  print(m.I_min[(u,v)])


Second scenario: envelopes given to DSOs

In [ ]:

info_DSO_node1 = sum(m.P[n].value for n in G.subgraph([5, 9, 10, 11,12]).nodes)
info_DSO_node2 = sum(m.P[n].value for n in G.subgraph([6,7,8,13]).nodes)
print(info_DSO_node1)
print(info_DSO_node2)

info_DSO = [info_DSO_node1, info_DSO_node2]

In [ ]:
"""info_DSO_node1 = m.F[4, 5, 0, 0].value
info_DSO_node2 = m.F[3, 8, 0, 0].value
print(info_DSO_node1)
print(info_DSO_node2)

info_DSO = [info_DSO_node1, info_DSO_node2]"""

In [ ]:
for n in m.Nodes:
    for vert_pow in m.i:
        for vert_volt in m.j:
            print(f"m.theta[{n},{vert_pow},{vert_volt}].value: {m.theta[n,vert_pow,vert_volt].value}")

In [ ]:
def plot_power_flow(m, i, j):
    pos = nx.get_node_attributes(G, 'pos')
    # Use node indices as labels
    labels = {n: n for n in G.nodes()}

    plt.figure(figsize=(12, 8))

    edge_colors = []
    edge_labels = {}
    # Create a list of edges with direction based on flow
    directed_edges = []


    for u, v in G.edges():
        try:
            # Get the flow value from the model
            flow_value = m.F[u, v, i, j].value
            if flow_value is not None:
                edge_labels[(u, v)] = f"{round(abs(flow_value), 4)}" # Display absolute value
                if flow_value > 0:
                    edge_colors.append('blue')  # Positive flow (u to v) - Blue arrow
                    directed_edges.append((u, v)) # Arrow from u to v
                elif flow_value < 0:
                    edge_colors.append('blue')  # Negative flow (v to u) - Still blue, but direction will be reversed
                    directed_edges.append((v, u)) # Arrow from v to u (reverse direction)
                else:
                    edge_colors.append('gray') # No flow
                    directed_edges.append((u, v)) # Still add edge for drawing, no specific direction
            else:
                edge_colors.append('gray') # No flow value
                directed_edges.append((u, v)) # Still add edge for drawing, no specific direction
        except Exception as e: # Catching the exception
            print(f"Error processing edge ({u}, {v}): {e}")
            edge_colors.append('gray') # Handle cases where edge might not be in m.F
            directed_edges.append((u, v)) # Still add edge for drawing, no specific direction


    # Create a directed graph for drawing purposes with correct edge directions
    draw_graph = nx.DiGraph()
    draw_graph.add_nodes_from(G.nodes(data=True))
    draw_graph.add_edges_from(directed_edges)

    # Draw the network
    nx.draw(
        draw_graph, pos,
        with_labels=True, labels=labels, # Draw labels with node indices
        node_size=1200,
        edge_color=edge_colors, # Use the calculated edge colors (now always blue or gray for non-zero flow)
        edgecolors="black", font_size=8,
        alpha=0.85,
        node_color='steelblue', # Default node color
        arrows=True, arrowsize=20, # Enable arrows and set size
        connectionstyle='arc3,rad=0.8' # Add some curvature to distinguish directions if needed
    )


    nx.draw_networkx_edge_labels(draw_graph, pos, edge_labels=edge_labels, font_size=7, label_pos=0.3) # Use draw_graph for edge labels

    plt.title(f"Power Flow (per-unit) for i={i}, j={j}, alpha = 3")
    plt.axis("equal")
    plt.show()

# Example usage (assuming m, i=0, and j=0 are defined)
plot_power_flow(m, 0, 1)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

"""# Extract all values of m.theta
theta_values = []
for n in m.Nodes:
    for vert_pow in m.i:
        for vert_volt in m.j:
            theta_values.append({
                'node': n,
                'i_index': vert_pow,
                'j_index': vert_volt,
                'theta_value': m.theta[n, vert_pow, vert_volt].value
            })

theta_df = pd.DataFrame(theta_values)
print("Extracted m.theta values:")
display(theta_df) # Display the first few rows"""

# Extract all values of m.I
i_values = []
for line in m.Lines:
    u, v = line
    for vert_pow in m.i:
        for vert_volt in m.j:
            i_values.append({
                'from_node': u,
                'to_node': v,
                'i_index': vert_pow,
                'j_index': vert_volt,
                'i_value': m.I[u, v, vert_pow, vert_volt].value
            })

i_df = pd.DataFrame(i_values)
print("\nExtracted m.I values:")
display(i_df) # Display the first few rows

In [ ]:
def create_graph(net) -> nx.Graph:
    import json
    # -------------------------
    # 1. Conversion du réseau Pandapower en DiGraph
    # -------------------------
    G = nx.Graph()

    # 2. Récupération des positions à partir de la colonne 'geo'
    pos = {}
    for idx, row in net.bus.iterrows():
        geo_data = row['geo']
        if geo_data is None:
            raise ValueError(f"Le bus '{row['name']}' (index {idx}) n'a pas de coordonnées 'geo'.")
        geo_dict = json.loads(geo_data)
        coordinates = geo_dict["coordinates"]
        pos[idx] = tuple(coordinates)

    #Ajouter une puissance de base
    G.graph["s_base"] = 100.0 # MVA
    G.graph["v_base"] = 110.0 # kV

    # Ajouter les nœuds
    for idx, row in net.bus.iterrows():
        G.add_node(idx,
                   label=row["name"],
                   pos=pos[idx],
                   vn_kv=row["vn_kv"])

    # Ajouter les arêtes pour les lignes
    for _, row in net.line.iterrows():
        G.add_edge(row["from_bus"], row["to_bus"],
                   type="line",
                   name=row["name"],
                   length=row["length_km"],
                   std_type=row["std_type"],
                   x_ohm=row["x_ohm_per_km"],)

    # Ajouter les arêtes pour les transformateurs
    for _, row in net.trafo.iterrows():
        G.add_edge(row["hv_bus"], row["lv_bus"],
                   type="trafo", std_type = None,
                   name=row["name"])

    # Ajouter les générateurs et les charges comme attributs aux nœuds
    for _, row in net.gen.iterrows():
        G.nodes[row["bus"]]["type"] = "gen"
        G.nodes[row["bus"]]["gen_name"] = row["name"]
        G.nodes[row["bus"]]["gen_power"] = row["p_mw"]

    for _, row in net.load.iterrows():
        G.nodes[row["bus"]]["type"] = "load"
        G.nodes[row["bus"]]["load_name"] = row["name"]
        G.nodes[row["bus"]]["load_power"] = row["p_mw"]

    for _, row in net.ext_grid.iterrows():
        G.nodes[row["bus"]]["type"] = "ext_grid"
        G.nodes[row["bus"]]["grid_name"] = row["name"]

    # -------------------------
    # 2. Define subnetworks and add power attributes to nodes
    # -------------------------
    network1_buses = [0, 1, 2, 3, 4]
    # network2_buses = [3, 7, 5, 9, 10]
    # network3_buses = [4, 8, 6]

    # Assign G1 to G and then add power attributes to nodes in G
    # Revert to the original subgraph definition
    G = G.subgraph(network1_buses)

    # Manually define the 'P' attribute for each node in the subgraph G
    # This replaces the calculation of P_load, P_gen, and P_net from the 'net' object
    # You should define the 'P' value for each node in network1_buses here.
    # Example:

    co = 4
    power_demand = {
        0: 0.0/co,
        1: 50,
        2: -20,
        3: 0/co,
        4: 0/co,
        5: -1e-1/co,
        6: 0.1/co,
        7: -0.2/co,
        8: -29.5*1e-2 /co,
        9: -9.0*1e-1 /co,
        10: -3.5*1e-1 /co,
        11: -6.1*1e-1 /co,
        12: -13.5*1e-2 /co,
        13: -14.9*1e-2 /co
    }


    for n in G.nodes:
        G.nodes[n]["P"] = power_demand.get(n, 0.0)
        G.nodes[n]["P_pu"] = G.nodes[n].get("P", 0.0) / s_base


    print(net.line.columns)

    # Calculate the susceptance and max_current of each line in Siemens
    for u, v, data in G.edges(data=True):
        if data.get('type') == 'trafo':
             # Assign a high max_current for transformers
             data['max_current_pu'] = 1000 # Use a large value
             print(f"Transformer {u}->{v}: max_current_pu = {data['max_current_pu']} pu (high value)")
        else:
            data['b_pu'] = (G.nodes[u]["vn_kv"]**2 / (data['x_ohm'] * G.graph["s_base"]))  # Calcule et stocke B_ij per unit
            print(f"Ligne {u}->{v}: b_pu = {G[u][v]['b_pu']} pu")

            # Calculate max_current for lines based on line type and node voltage
            line_type = data.get('std_type')
            v_node_kv = G.nodes[u].get("vn_kv") # Get voltage of the 'from' node

            if line_type and v_node_kv is not None:
                I_min_pu, I_max_pu = calculate_current_bounds(v_node_kv)
                if I_max_pu is not None:
                    data['max_current_pu'] = I_max_pu
                    print(f"Line {u}->{v}: max_current_pu = {data['max_current_pu']} pu")
                else:
                    print(f"Warning: Could not calculate max_current for line {u}->{v} (unknown line type or voltage).")
                    data['max_current_pu'] = None # Or a default value if appropriate
            else:
                 print(f"Warning: Missing line type or node voltage for line {u}->{v}. Could not calculate max_current.")
                 data['max_current_pu'] = None # Or a default value if appropriate


    B_base = G.graph["s_base"]/(G.graph["v_base"]**2)
    print("B_base: ", B_base)

    # -------------------------
    # Donner accès à G
    # -------------------------
    node_attrs = {n: G.nodes[n] for n in G.nodes}
    return G # Return G and the plot_network function

In [ ]:
G = create_graph(net)

In [ ]:
plot_network(G)

In [ ]:
m = pyo.ConcreteModel()
# Define base values for the per-unit system
s_base = 100  # MVA
v_base_high = 110 # kV (From the create_network function)
first_child = 3
m.Nodes = pyo.Set(initialize= [b for b in G.nodes])
m.Lines = pyo.Set(initialize= [l for l in G.edges])
m.i = pyo.Set(initialize=[0, 1]) # Initialize m.i with two generic elements
m.j = pyo.Set(initialize= [0, 1])
m.children = pyo.Set(initialize= [first_child, first_child + 1])
m.parents = pyo.Set(initialize = [0])

m.P = pyo.Param(m.Nodes, initialize = {n: - G.nodes[n].get('P_pu') for n in G.nodes}, domain = pyo.Reals, mutable=True) # Make P mutable
m.P_absolute_value = pyo.Param(m.Nodes, initialize = {n: abs(m.P[n]) for n in G.nodes}, domain = pyo.NonNegativeReals) # Make P mutable

m.info_DSO_param = pyo.Param(m.children, initialize = {n: info_DSO[n-first_child] for n in m.children}, domain = pyo.Reals) # Renamed parameter and adjusted index for list

print(m.info_DSO_param[3])
print(m.info_DSO_param[4])


m.I_min = pyo.Param(m.Lines, initialize = {(i,j): calculate_current_bounds(G.nodes[i]["vn_kv"])[0] for (i,j) in m.Lines}, domain = pyo.Reals)
m.I_max = pyo.Param(m.Lines, initialize = {(i,j): calculate_current_bounds(G.nodes[i]["vn_kv"])[1] for (i,j) in m.Lines}, domain = pyo.Reals)

print(m.I_min[(0,1)])
print(m.I_max[(0,1)])

# Variables principales
m.F = pyo.Var(m.Lines, m.i, m.j, domain=pyo.Reals)     # active power flow through lines
m.I = pyo.Var(m.Lines,  m.i, m.j, domain=pyo.Reals)  # current flowing through lines
m.theta = pyo.Var(m.Nodes,  m.i, m.j, domain=pyo.Reals)  # phase angle of the voltage
m.V = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.NonNegativeReals)  # voltage magnitude at each node
m.E = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.Reals) #net power injection/consumption
m.P_plus = pyo.Var(m.parents, m.i, m.j, domain=pyo.Reals) #power entering the operational graph
m.P_minus = pyo.Var(m.children, m.i, m.j, domain=pyo.Reals) #power leaving the operational graph

m.P_C_set = pyo.Var(m.children, m.i, domain=pyo.Reals) #vertices of the power envelope at each child node

#m.P_C_min = pyo.Var(m.children, m.i, m.j, domain=pyo.Reals)
#m.P_C_max = pyo.Var(m.children, m.i, m.j,  domain=pyo.Reals)

m.V_P = pyo.Param(m.j, initialize = {0: 0.9, 1: 1.1}, domain =pyo.NonNegativeReals)

m.O = pyo.Var(domain= pyo.NonNegativeReals)
m.tot = pyo.Var(domain= pyo.Reals)
m.aux = pyo.Var(m.children, domain=pyo.Reals)
m.diff_DSO = pyo.Var(m.children, domain=pyo.NonNegativeReals)

for u in G.nodes():
  if G.nodes[u].get('P', 0.0) / s_base == 0:
    m.P[u] = 0
  else:
    G.nodes[u]['P_pu'] = G.nodes[u].get('P', 0.0) / s_base
    m.P[u] = - G.nodes[u]['P_pu']

"""# Convert P parameter to per-unit
for n in G.nodes:
    # Assuming P is in MW, convert to per-unit
    G.nodes[n]['P'] = G.nodes[n].get('P', 0.0) / s_base
    m.P[n] = G.nodes[n]['P']"""

In [ ]:
for n in G.nodes:
  print(m.P[n].value)
m.P[1].value


In [ ]:
for u, v, data in G.edges(data=True):
    if 'b_pu' in data:
        print(f"Line ({u}, {v}): b_pu = {data['b_pu']} pu")
    else:
        print(f"Line ({u}, {v}): b_pu not available")

In [ ]:
#Children nodes consumption
def worst_case_children(m, n, vert_pow, vert_volt):
  return m.P_C_set[n, vert_pow] == m.P_minus[n, vert_pow, vert_volt]
m.worst_case = pyo.Constraint(m.children, m.i, m.j, rule=worst_case_children)


# Auxiliary variable for the absolute value of E (already defined as per-unit)
m.z = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.NonNegativeReals)

m.curt = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.Reals)

def curtailment_def_rule(m,n, vert_pow, vert_volt):

  return m.curt[n, vert_pow, vert_volt] == m.P[n] - m.E[n, vert_pow, vert_volt]

m.curt_def = pyo.Constraint(m.Nodes, m.i, m.j, rule=curtailment_def_rule)
# Constraints to define the absolute value (already defined for per-unit E)
def abs_E_pos_rule(m, n, vert_pow, vert_volt):
    return m.z[n, vert_pow, vert_volt] >= m.curt[n, vert_pow, vert_volt]
m.abs_E_pos = pyo.Constraint(m.Nodes, m.i, m.j, rule=abs_E_pos_rule)

def abs_E_neg_rule(m, n, vert_pow, vert_volt):
    return m.z[n, vert_pow, vert_volt] >= -m.curt[n, vert_pow, vert_volt]
m.abs_E_neg = pyo.Constraint(m.Nodes, m.i, m.j, rule=abs_E_neg_rule)

def upper_bound_rule(m, vert_pow, vert_volt):
    # m.z is per-unit, m.O is per-unit
    return sum(m.z[n, vert_pow, vert_volt] for n in m.Nodes) <= m.O

m.upper_bound = pyo.Constraint(m.i, m.j, rule=upper_bound_rule)

# Current magnitude constraint (I_min, I_max are assumed per-unit)
def current_bounds_rule(m, i, j, vert_pow, vert_volt):
    # m.I is per-unit current
    return pyo.inequality(m.I_min[(i,j)], m.I[i,j, vert_pow, vert_volt], m.I_max[(i,j)])
m.CurrentBounds = pyo.Constraint(m.Lines, m.i, m.j, rule=current_bounds_rule)

def phase_constr_rule(m,n, vert_pow, vert_volt):
    return pyo.inequality(theta_min, m.theta[n, vert_pow, vert_volt], theta_max)

m.phaseConstr = pyo.Constraint(m.Nodes, m.i, m.j, rule=phase_constr_rule)

def dc_power_flow_rule(m, i, j, vert_pow, vert_volt):
    if G[i][j].get('b_pu') is None:
      return pyo.Constraint.Skip
    else:
      return m.F[i, j, vert_pow, vert_volt] == m.V_P[vert_volt]**2 *(G[i][j]['b_pu'] * (
        m.theta[i, vert_pow, vert_volt] - m.theta[j, vert_pow, vert_volt])
    )

m.DCFlow = pyo.Constraint(m.Lines, m.i, m.j, rule=dc_power_flow_rule)

def current_def_rule(m, n, i, j, vert_pow, vert_volt):
    # This constraint relates per-unit current, per-unit voltage, and per-unit power flow.
    # In per-unit, P_pu = V_pu * I_pu. This is correct.
    return np.sqrt(3)* m.I[i,j, vert_pow, vert_volt] * m.V_P[vert_volt] == m.F[i,j, vert_pow, vert_volt]

m.current_def = pyo.Constraint(m.Nodes, m.Lines, m.i, m.j, rule=current_def_rule)



"""def nodes_balance(m, n, vert_pow, vert_volt):
    inflow = sum(m.F[k, n, vert_pow, vert_volt] for k in G.predecessors(n) if (k, n) in m.Lines)
    outflow = sum(m.F[n, j, vert_pow, vert_volt] for j in G.successors(n) if (n, j) in m.Lines)
    if n in m.parents:
      return inflow - outflow  ==  m.E[n, vert_pow, vert_volt] - m.P_plus[n, vert_pow, vert_volt]

    if n in m.children:
      return inflow - outflow  ==  m.E[n, vert_pow, vert_volt] + m.P_minus[n, vert_pow, vert_volt]

    else:
      return inflow - outflow ==  m.E[n,vert_pow,vert_volt]

m.nodes_balance = pyo.Constraint(m.Nodes, m.i, m.j, rule=nodes_balance)"""

def power_balance_rule(m, n, vert_pow, vert_volt):
    # Compute net flow into node n by summing over all lines (i,j) in m.Lines
    expr = sum(
        (m.F[i, j, vert_pow, vert_volt] if j == n else 0)
      - (m.F[i, j, vert_pow, vert_volt] if i == n else 0)
      for (i, j) in m.Lines
    )
    # If n is a parent node, subtract P_plus; otherwise use only E[n]
    if n in m.parents:
      return expr == m.E[n, vert_pow, vert_volt] - m.P_plus[n, vert_pow, vert_volt]

    if n in m.children:
      return expr ==  m.E[n, vert_pow, vert_volt] + m.P_minus[n, vert_pow, vert_volt]
    else:
      return expr == m.E[n, vert_pow, vert_volt]

m.power_balance = pyo.Constraint(m.Nodes, m.i, m.j, rule=power_balance_rule)

def parent_power_constraint_rule(m, parent, vert_pow, vert_volt):
    # m.P_plus is per-unit power entering the operational graph
    return pyo.inequality(P_min, m.P_plus[parent, vert_pow, vert_volt],P_max)

m.parent_power_constraint = pyo.Constraint(m.parents, m.i, m.j, rule=parent_power_constraint_rule)


#Constant voltage assumption
def voltage_constr_rule(m, n, vert_pow, vert_volt):
    return m.V[n, vert_pow, vert_volt] == m.V_P[vert_volt]

m.voltageConstr = pyo.Constraint(m.Nodes, m.i, m.j, rule=voltage_constr_rule)

def children_voltage_rule(m, children, vert_pow, vert_volt):
    return pyo.inequality(V_min, m.V[children, vert_pow, vert_volt], V_max)

m.children_voltage = pyo.Constraint(m.children, m.i, m.j, rule=children_voltage_rule)



def aux_constraint_rule(m, n):
  return m.aux[n] == m.P_C_set[n, 0] - m.P_C_set[n, 1]
m.aux_constraint = pyo.Constraint(m.children, rule=aux_constraint_rule)

def logical_constraint_rule(m, n):
  return m.P_C_set[n, 0] >= m.P_C_set[n, 1]
m.logical_constraint = pyo.Constraint(m.children, rule=logical_constraint_rule)

"""def info_DSO_min_rule(m,n):
  return m.P_C_set[n,1] <= m.info_DSO_param[n]
m.info_DSO_min = pyo.Constraint(m.children, rule=info_DSO_min_rule)

def info_DSO_max_rule(m,n):
  return m.P_C_set[n,0] >= m.info_DSO_param[n]
m.info_DSO_max = pyo.Constraint(m.children, rule=info_DSO_max_rule)"""

def tot_rule(m):
  return m.tot == sum(m.aux[n] for n in m.children)
m.tot_constraint = pyo.Constraint(rule=tot_rule)

def diff_DSO_rule(m,n):
  return - m.diff_DSO[n] <= (m.P_C_set[n,0] + m.P_C_set[n,1])/2 - m.info_DSO_param[n]
m.diff_DSO_constraint = pyo.Constraint(m.children, rule=diff_DSO_rule)

def diff_bis_dso_rule(m,n):
  return (m.P_C_set[n,0] + m.P_C_set[n,1])/2 - m.info_DSO_param[n] <= m.diff_DSO[n]
m.diff_bis_dso_constraint = pyo.Constraint(m.children, rule=diff_bis_dso_rule)


m.abs_diff_DSO = pyo.Var(m.children, domain=pyo.NonNegativeReals)
"""def abs_different_rule(m, n):
    return  m.abs_diff_DSO[n] >= -m.diff_DSO[n]
m.abs_different = pyo.Constraint(m.children, rule=abs_different_rule)

def abs_different_bis_rule(m, n):
    return  m.abs_diff_DSO[n] >= m.diff_DSO[n]
m.abs_different = pyo.Constraint(m.children, rule=abs_different_rule)"""

m.tot_bis = pyo.Var(domain=pyo.Reals)
def tot_bis_rule(m):
  return m.tot_bis == sum(m.diff_DSO[n] for n in m.children)
m.tot_bis_constraint = pyo.Constraint(rule=tot_bis_rule)

# Build sets based on sign of parameter P
m.PositiveNodes = pyo.Set(initialize=[n for n in m.Nodes if pyo.value(m.P[n]) >= 0])
m.NegativeNodes = pyo.Set(initialize=[n for n in m.Nodes if pyo.value(m.P[n]) <= 0])

# Constraint for positive P: upper bound
def net_power_upper_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] <= m.P[n]
m.net_power_upper = pyo.Constraint(m.PositiveNodes, m.i, m.j, rule=net_power_upper_rule)

def sign_E_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] >= 0

m.sign_E = pyo.Constraint(m.PositiveNodes, m.i, m.j, rule=sign_E_rule)



# Constraint for negative P: lower bound
def net_power_lower_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] >= m.P[n]
m.net_power_lower = pyo.Constraint(m.NegativeNodes, m.i, m.j, rule=net_power_lower_rule)

def sign_E_lower_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] <= 0

m.sign_E_lower = pyo.Constraint(m.NegativeNodes, m.i, m.j, rule=sign_E_lower_rule)

m.positive_demand = pyo.Set(initialize=[n for n in m.children if pyo.value(m.info_DSO_param[n]) >= 0])
m.negative_demand = pyo.Set(initialize=[n for n in m.children if pyo.value(m.info_DSO_param[n]) <= 0])

def p_C_plus_limit_rule(m, pos):
    return m.P_C_set[pos, 1] <= 100

m.p_C_plus_limit = pyo.Constraint(m.positive_demand, rule=p_C_plus_limit_rule)

"""def p_C_minus_limit_rule(m, neg):
    return m.P_C_set[neg, 0] >= m.info_DSO_param[neg]

m.p_C_minus_limit = pyo.Constraint(m.negative_demand, rule=p_C_minus_limit_rule)"""

In [ ]:
A = np.arange(0.5, 2.6, 0.1)
beta = 1
enveloppe_taille = []
curtail = []
close = []

somme_contributions = []
for alpha in A:
  val = 0
  print(alpha)
  # Check if P_C_set has been initialized before accessing its values
  if hasattr(m, 'P_C_set') and len(m.children) > 0:
      try:
          print(m.P_C_set[first_child,0].value)
          print(m.P_C_set[first_child,1].value)
          print(m.P_C_set[first_child + 1,0].value)
          print(m.P_C_set[first_child +1,1].value)
      except KeyError as e:
          print(f"Warning: Could not access P_C_set for child node {e}. Skipping print.")


  def objective_rule(m):
    return  m.tot - alpha*m.O - beta * m.tot_bis
  m.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)
  params = {
    "WLSACCESSID": 'efa10779-150a-495c-ab62-6a20ff44c598',
    "WLSSECRET": 'fa980d94-a6fd-4d7b-aa68-b0e19ddd2e84',
    "LICENSEID": 2651433,
           }
  env = gp.Env(params=params)
  solver = pyo.SolverFactory('gurobi', env= env)
  results = solver.solve(m, tee=True)

  # Check the termination condition before accessing results
  if results.solver.termination_condition == pyo.TerminationCondition.optimal:
      val = sum(abs(m.P_C_set[n,0].value) + abs(m.P_C_set[n,1].value) for n in m.children)
      # Append optimal results to lists
      curtail.append(m.O.value)
      enveloppe_taille.append(m.tot.value)
      close.append(m.tot_bis.value)
      somme_contributions.append(m.tot.value + m.tot_bis.value)

  else:
      # Append None for non-optimal results to maintain list length
      curtail.append(None)
      enveloppe_taille.append(None)
      close.append(None)
      somme_contributions.append(None)

In [ ]:
print(enveloppe_taille)
print(curtail)
print(close)
print(somme_contributions)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple

# Convert A to a numpy array for easier plotting
alpha_values = np.array(A)

# Convert lists to numpy arrays, replacing None with NaN for plotting
enveloppe_taille_np = np.array(enveloppe_taille, dtype=float)
curtail_np = np.array(curtail, dtype=float)
close_np = np.array(close, dtype=float)

plt.figure(figsize=(10, 6))

# Envelope Volume (BLUE)
plt.plot(
    alpha_values, enveloppe_taille_np,
    marker='o', linestyle='-', color='blue',
    label='Envelope Volume'
)

# Curtailment (ORANGE)
plt.plot(
    alpha_values, curtail_np,
    marker='x', linestyle='--', color='orange',
    label='Curtailment'
)

# Deviation from DSO (GREEN)
plt.plot(
    alpha_values, close_np,
    marker='s', linestyle='--', color='green',
    label='Deviation of the center of the envelope from DSO estimation'
)

# --- Alternating-color sum curve: blue (envelope) ↔ green (deviation) ---
alt_colors = cycle(['blue', 'green'])
for i in range(len(alpha_values) - 1):
    plt.plot(
        alpha_values[i:i+2],
        somme_contributions[i:i+2],
        linestyle=':',
        linewidth=1.2,
        alpha=0.9,
        color=next(alt_colors),
        zorder=2
    )

# --- Custom legend entry for alternating curve (overlay, no gap) ---
custom_sum_handle = (
    Line2D([0, 1], [0, 0], color='blue', linestyle=':', linewidth=1.5),
    Line2D([0, 1], [0, 0], color='green', linestyle=':', linewidth=1.5)
)

# Collect existing legend entries
handles, labels = plt.gca().get_legend_handles_labels()

# Add custom alternating-color entry
handles.append(custom_sum_handle)
labels.append('Deviation of the center of the envelope from DSO estimation + Envelope Volume')

# Place legend below the plot, centered
plt.legend(
    handles, labels,
    handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
    loc='upper center',
    bbox_to_anchor=(0.5, -0.15),  # move legend below plot
    ncol=2,  # spread over 2 columns (adjust if needed)
    frameon=False,
    fontsize='large' # Increase legend font size
)

# Axis formatting
plt.xlabel('$\\alpha$', fontsize='large')
plt.ylabel('Power (per-unit)', fontsize='large')
plt.grid(True)

# Adjust layout so legend fits underneath
plt.tight_layout(rect=[0, 0.05, 1, 1])

# Save and show
plt.savefig('DOE_alloc_alpha_final.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Print the values of m.E
for n in m.Nodes:
  for vert_pow in m.i:
    for vert_volt in m.j:
      print(f"m.E[{n},{vert_pow},{vert_volt}].value: {m.E[n,vert_pow,vert_volt].value}")

# Get the values of m.P for plotting
p_values = [m.P[n].value for n in m.Nodes]
node_indices = list(m.Nodes)

# Create a bar plot of m.P values
plt.figure(figsize=(10, 6))
plt.bar(node_indices, p_values, color='skyblue')
plt.xlabel('Node Index')
plt.ylabel('P (per-unit)')
plt.title('Values of m.P at Each Node')
plt.xticks(node_indices) # Ensure all node indices are shown on the x-axis
plt.grid(axis='y', linestyle='--')
plt.show()

In [ ]:
m.O.value

In [ ]:
print(m.P_C_set[first_child,0].value)
print(m.P_C_set[first_child,1].value)

print(m.P_C_set[first_child + 1,0].value)
print(m.P_C_set[first_child + 1,1].value)

In [ ]:
print(m.theta[0,0,0].value)
print(m.theta[0,0,1].value)

In [ ]:

m.O.value


In [ ]:
for (u,v) in m.Lines:
  print((u,v))

In [ ]:
# Corrected loop structure to iterate over m.Lines
for (u, v) in m.Lines:
  for vert_pow in m.i:
    for vert_volt in m.j:
      print(f"Flow on line ({u}, {v}) for i={vert_pow}, j={vert_volt}: {m.F[u, v, vert_pow, vert_volt].value}")

In [ ]:
def plot_power_flow(m, i, j):
    pos = nx.get_node_attributes(G, 'pos')
    # Use node indices as labels
    labels = {}
    label_colors = [] # This is for node colors, will remove this later if needed or set to default
    # Use the defined children and parents sets from the model
    children_nodes_set = m.children
    parents_nodes_set = m.parents

    for n in G.nodes():
        label_text = f"{n}"
        if n in parents_nodes_set:
            # Display parent bounds using the global P_min and P_max parameters
            label_text += f"\n[{P_min}, {P_max}]"
            # No specific color for label text here, use default
            label_colors.append('steelblue') # Default node color based on previous plots
        elif n in children_nodes_set:
            # Display children interval with smaller value first
            # Access P_C_set values - assuming P_C_set is defined and indexed by children and i
            p_c_values = [m.P_C_set[n, 0].value, m.P_C_set[n, 1].value] # Assuming m.P_C_set is indexed by children and i
            label_text += f"\n[{round(min(p_c_values), 4)}, {round(max(p_c_values), 4)}]"
            # We will try to color this text red when drawing labels
            label_colors.append('steelblue') # Default node color
        else:
            label_colors.append('steelblue') # Default node color
        labels[n] = label_text


    plt.figure(figsize=(12, 8))

    edge_colors = []
    edge_labels = {}
    # Create a list of edges with direction based on flow
    directed_edges = []


    for u, v in G.edges():
        try:
            # Get the flow value from the model
            flow_value = m.F[u, v, i, j].value
            if flow_value is not None:
                edge_labels[(u, v)] = f"{round(abs(flow_value), 4)}" # Display absolute value
                if flow_value > 0:
                    edge_colors.append('blue')  # Positive flow (u to v) - Blue arrow
                    directed_edges.append((u, v)) # Arrow from u to v
                elif flow_value < 0:
                    edge_colors.append('blue')  # Negative flow (v to u) - Still blue, but direction will be reversed
                    directed_edges.append((v, u)) # Arrow from v to u (reverse direction)
                else:
                    edge_colors.append('gray') # No flow
                    directed_edges.append((u, v)) # Still add edge for drawing, no specific direction
            else:
                edge_colors.append('gray') # No flow value
                directed_edges.append((u, v)) # Still add edge for drawing, no specific direction
        except Exception as e: # Catching the exception
            print(f"Error processing edge ({u}, {v}): {e}")
            edge_colors.append('gray') # Handle cases where edge might not be in m.F
            directed_edges.append((u, v)) # Still add edge for drawing, no specific direction


    # Create a directed graph for drawing purposes with correct edge directions
    draw_graph = nx.DiGraph()
    draw_graph.add_nodes_from(G.nodes(data=True))
    draw_graph.add_edges_from(directed_edges)

    # Draw the network
    nx.draw(
        draw_graph, pos,
        with_labels=False, # Draw labels separately for color control
        node_size=1200,
        edge_color=edge_colors, # Use the calculated edge colors (now always blue or gray for non-zero flow)
        edgecolors="black", font_size=8,
        alpha=0.85,
        node_color = label_colors # Apply node colors
    )

    # Draw labels with different colors
    for n in G.nodes():
        x, y = pos[n]
        text = labels[n]
        if n in children_nodes_set:
            plt.text(x, y - 0.1, text, fontsize=10, ha='center', va='top', color='red') # Color children interval red
        elif n in parents_nodes_set:
             plt.text(x, y + 0.1, text, fontsize=10, ha='center', va='bottom', color='black') # Color parent bounds black
        else:
             plt.text(x, y, text, fontsize=8, ha='center', va='center', color='black') # Default color for other labels


    nx.draw_networkx_edge_labels(draw_graph, pos, edge_labels=edge_labels, font_size=7, label_pos=0.3) # Use draw_graph for edge labels

    plt.title(f"Power Flow (per-unit) for i={i}, j={j}, alpha = 3")
    plt.axis("equal")
    plt.show()

# Example usage (assuming m, i=0, and j=0 are defined)
plot_power_flow(m, 0, 1)

In [ ]:
def plot_current_flow(m, i, j):
    pos = nx.get_node_attributes(G, 'pos')
    # Use node indices as labels
    labels = {n: n for n in G.nodes()}

    plt.figure(figsize=(12, 8))

    edge_colors = []
    edge_labels = {}

    for u, v in G.edges():
        try:
            # Get the current value from the model
            current_value = m.I[u, v, i, j].value
            if current_value is not None:
                edge_labels[(u, v)] = f"{round(current_value, 4)}"
                if current_value > 0:
                    edge_colors.append('blue')  # Positive current flow
                elif current_value < 0:
                    edge_colors.append('red')  # Negative current flow
                else:
                    edge_colors.append('gray') # No current flow
            else:
                edge_colors.append('gray') # No current value
        except:
            edge_colors.append('gray') # Handle cases where edge might not be in m.I

    nx.draw(
        G, pos,
        with_labels=True, labels=labels, # Set with_labels to True and pass node indices as labels
        node_size=1200,
        edge_color=edge_colors, # Use the calculated edge colors
        edgecolors="black", font_size=8,
        alpha=0.85
    )

    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7, label_pos=0.3)

    plt.title(f"Current Flow (per-unit) for i={i}, j={j}")
    plt.axis("equal")
    plt.show()

# Example usage (assuming m, i=0, and j=0 are defined)
plot_current_flow(m, 0, 0)

In [ ]:
B = np.arange(0.5, 2.6, 0.1) # Iterate from 0.5 to 3.2 with a step of 0.1
#A = range(-3, 26)
alpha = 1.5
enveloppe_taille = []
curtail = []
close = []
somme_contributions = []

for beta in B:
  print(alpha)
  def objective_rule(m):
    return  m.tot - alpha*m.O - beta * m.tot_bis
  m.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)
  params = {
    "WLSACCESSID": 'efa10779-150a-495c-ab62-6a20ff44c598',
    "WLSSECRET": 'fa980d94-a6fd-4d7b-aa68-b0e19ddd2e84',
    "LICENSEID": 2651433,
           }
  env = gp.Env(params=params)
  solver = pyo.SolverFactory('gurobi', env= env)
  results = solver.solve(m, tee=True)

  # Check the termination condition before accessing results
  if results.solver.termination_condition == pyo.TerminationCondition.optimal:
      curtail.append(m.O.value)
      enveloppe_taille.append(m.tot.value)
      close.append(m.tot_bis.value)
      somme_contributions.append(m.O.value + m.tot.value)

  else:
      # Append None for non-optimal results to maintain list length
      curtail.append(None)
      enveloppe_taille.append(None)
      close.append(None)

In [ ]:
print(enveloppe_taille)
print(curtail)
print(close)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple

# Convert B to a numpy array for easier plotting
alpha_values = np.array(B)

# Convert lists to numpy arrays, replacing None with NaN for plotting
enveloppe_taille_np = np.array(enveloppe_taille, dtype=float)
curtail_np = np.array(curtail, dtype=float)
close_np = np.array(close, dtype=float)

plt.figure(figsize=(10, 6))

# Envelope Volume (BLUE)
plt.plot(
    alpha_values, enveloppe_taille_np,
    marker='o', linestyle='-', color='blue',
    label='Envelope Volume'
)

# Curtailment (ORANGE)
plt.plot(
    alpha_values, curtail_np,
    marker='x', linestyle='--', color='orange',
    label='Curtailment'
)

# Deviation (GREEN)
plt.plot(
    alpha_values, close_np,
    marker='s', linestyle='-.', color='green',
    label='Deviation of the center of the envelope from DSO estimation'
)

# --- Alternating-color sum curve: Curtailment (orange) + Envelope (blue) ---
alt_colors = cycle(['orange', 'blue'])
for i in range(len(alpha_values) - 1):
    plt.plot(
        alpha_values[i:i+2],
        somme_contributions[i:i+2],
        linestyle=':',
        linewidth=1.2,
        alpha=0.9,
        color=next(alt_colors),
        zorder=2
    )

# --- Custom legend entry for alternating curve (overlay, no gap) ---
custom_sum_handle = (
    Line2D([0, 1], [0, 0], color='orange', linestyle=':', linewidth=1.5),
    Line2D([0, 1], [0, 0], color='blue', linestyle=':', linewidth=1.5)
)

# Collect existing legend entries
handles, labels = plt.gca().get_legend_handles_labels()

# Add custom alternating-color entry
handles.append(custom_sum_handle)
labels.append('Curtailment + Envelope Volume')

# Place legend below the plot, centered
plt.legend(
    handles, labels,
    handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
    loc='upper center',
    bbox_to_anchor=(0.5, -0.15),  # move legend below plot
    ncol=2,  # adjust # of columns depending on layout preference
    frameon=False,
    fontsize='large' # Increase legend font size
)

# Axes formatting
plt.xlabel('$\\beta$', fontsize='large')
plt.ylabel('Power (per-unit)', fontsize='large')
plt.grid(True)

# Adjust layout so legend fits underneath
plt.tight_layout(rect=[0, 0.05, 1, 1])

# Save and show
plt.savefig('DOE_alloc_beta_final.pdf', bbox_inches='tight')
plt.show()

In [ ]:
beta = 4
alpha = 2
enveloppe_taille = []
curtail = []
close = []


print(alpha)
def objective_rule(m):
  return  m.tot - alpha*m.O - beta * m.tot_bis
m.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)
params = {
    "WLSACCESSID": 'efa10779-150a-495c-ab62-6a20ff44c598',
    "WLSSECRET": 'fa980d94-a6fd-4d7b-aa68-b0e19ddd2e84',
    "LICENSEID": 2651433,
           }
env = gp.Env(params=params)
solver = pyo.SolverFactory('gurobi', env= env)
results = solver.solve(m, tee=True)

  # Check the termination condition before accessing results
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
  curtail.append(m.O.value)
  enveloppe_taille.append(m.tot.value)
  close.append(m.tot_bis.value)

else:

      # Append None for non-optimal results to maintain list length
  curtail.append(None)
  enveloppe_taille.append(None)
  close.append(None)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract data for plotting
children_nodes = list(m.children)
p_c_set_0_values = [m.P_C_set[n, 0].value for n in m.children]
p_c_set_1_values = [m.P_C_set[n, 1].value for n in m.children]
info_dso_values = [m.info_DSO_param[n] for n in m.children]

# Create the plot
plt.figure(figsize=(5, 6))

# Adjust x_pos to control spacing (e.g., reduce the multiplier)
x_pos = np.arange(len(children_nodes)) * 0.0005 # Reduced spacing by multiplying by 0.0005

# Plot lines between P_C_set[n][0] and P_C_set[n][1] for each child node
for i in range(len(children_nodes)):
    plt.plot([x_pos[i], x_pos[i]], [p_c_set_1_values[i], p_c_set_0_values[i]], marker='o', linestyle='-', color='blue')

# Plot only dots for info_DSO_param
plt.plot(x_pos, info_dso_values, marker='s', linestyle='', label=f'DSO power demand estimation')

# Create legend handles for the P_C_set endpoints
# We plot invisible points with the desired labels to create legend entries
plt.plot([], [], marker='o', linestyle='-', color='blue', label=f'Power envelope ($\\alpha$={alpha}, $\\beta$={beta})')


plt.xticks(x_pos, children_nodes)
plt.xlabel('Child Node Index')
plt.ylabel('Power (in per-unit)')
#plt.title('Comparison of P_C_set values and DSO estimation for Child Nodes')
plt.legend(loc='upper left') # Move the legend to the upper left
plt.grid(True)
# Ensure the y-axis is visible
plt.autoscale(enable=True, axis='y', tight=False)

# Adjust plot layout to prevent labels from being cut off
plt.tight_layout()


# Save the plot as a PDF file
plt.savefig('child_nodes_envelopes_TSO.pdf')

plt.show()

DOEs applied by DSOs

In [ ]:
min_power_DSO = []
max_power_DSO = []
for n in m.children:
  max_power_DSO.append(m.P_C_set[n,0].value)
  min_power_DSO.append(m.P_C_set[n,1].value)

print(min_power_DSO)
print(max_power_DSO)

In [ ]:
def create_digraph():
  net = pp.networks.case14()

  # Delete specified lines and transformer


  import json
  import networkx as nx
  import matplotlib.pyplot as plt
# 1. Conversion du réseau Pandapower en DiGraph
# ================================
  G = nx.Graph()

# Créer les positions à partir de la colonne "geo"
  pos = {}
  for idx, row in net.bus.iterrows():
  # Extraire le champ 'geo' qui est une chaîne de caractères représentant un dictionnaire JSON
      geo_data = row['geo']

  # Convertir la chaîne JSON en un dictionnaire Python
      geo_dict = json.loads(geo_data)

  # Extraire la liste des coordonnées
      coordinates = geo_dict["coordinates"]

  # Ajouter les coordonnées dans le dictionnaire pos
      pos[idx] = tuple(coordinates)  # Assigner les coordonnées sous forme de tuple

# Ajouter les nœuds (avec le nom comme label) and voltage
  for idx, row in net.bus.iterrows():
      G.add_node(idx, label=row["name"], pos=pos[idx], vn_kv=row["vn_kv"])


# Ajouter les arêtes pour les lines
  for _, row in net.line.iterrows():
      G.add_edge(row["from_bus"], row["to_bus"], type="line", name=row["name"], length=row["length_km"], std_type=row["std_type"])
      print(row["std_type"])

# Ajouter les arêtes pour les transformateurs
  for _, row in net.trafo.iterrows():
      G.add_edge(row["hv_bus"], row["lv_bus"], type="trafo", name=row["name"], std_type = None)

# Ajouter les générateurs et les charges comme attributs aux nœuds
  for _, row in net.gen.iterrows():
      G.nodes[row["bus"]]["type"] = "gen"
      G.nodes[row["bus"]]["gen_name"] = row["name"]
      G.nodes[row["bus"]]["gen_power"] = row["p_mw"]

  for _, row in net.load.iterrows():
      G.nodes[row["bus"]]["type"] = "load"
      G.nodes[row["bus"]]["load_name"] = row["name"]
      G.nodes[row["bus"]]["load_power"] = row["p_mw"]

  for _, row in net.ext_grid.iterrows():
      G.nodes[row["bus"]]["type"] = "ext_grid"
      G.nodes[row["bus"]]["grid_name"] = row["name"]

# 2. Ajout des puissances consommées et injectées aux nœuds
# ================================
  nx.set_node_attributes(G, 0.0, "P_load")
  nx.set_node_attributes(G, 0.0, "P_gen")

# Charges
  for _, row in net.load.iterrows():
      G.nodes[row["bus"]]["P_load"] += row["p_mw"]
      print(row)

# Générateurs
  for _, row in net.gen.iterrows():
      G.nodes[row["bus"]]["P_gen"] += row["p_mw"]

# Source externe
  for _, row in net.ext_grid.iterrows():
      G.nodes[row["bus"]]["P_gen"] += 0

# Calculate P_net
  for n in G.nodes:
      G.nodes[n]["P_net"] = G.nodes[n]["P_gen"] - G.nodes[n]["P_load"]

# Create subnetworks (NetworkX DiGraphs) from the main graph G
  #network1_buses = [0, 1, 2, 3,4]
  #network2_buses = [3, 6, 7, 8, 13]
  network3_buses = [4, 5, 9,10,11,12]

  G = G.subgraph(network3_buses)

  co = 4
  power_demand = {
        0: 0.0/co,
        1: -20,
        2: 50,
        3: 0/co,
        4: 0/co,
        5: -1e-1/co,
        6: 0.1/co,
        7: -0.2/co,
        8: -29.5*1e-2 /co,
        9: -9.0*1e-1 /co,
        10: -3.5*1e-1 /co,
        11: -6.1*1e-1 /co,
        12: -13.5*1e-2 /co,
        13: -14.9*1e-2 /co
    }
  for n in G.nodes:
      G.nodes[n]["P"] = power_demand[n] # Assign P_net to 'P' attribute
      G.nodes[n]["P_pu"] = G.nodes[n]["P"] / s_base

# 3. Préparer les couleurs des nœuds en fonction de P_net
# ================================
  node_colors = []
  for n, data in G.nodes(data=True):
      if data["P_net"] > 0:
          node_colors.append("green")   # producteur
      elif data["P_net"] < 0:
          node_colors.append("red")     # consommateur
      else:
          node_colors.append("gray")    # neutre

# 4. Préparer les labels : Nom + P_net
# ================================
  labels = {n: f"{data['label']}\nP={round(data['P_net'],2)}MW"
            for n, data in G.nodes(data=True)}

# 5. Function to plot the network
# ================================
  def plot_network():
      pos = nx.get_node_attributes(G, 'pos')

      plt.figure(figsize=(12, 8))
      nx.draw(
          G, pos,
          with_labels=True, labels=labels,
          node_size=1200, node_color=node_colors,
          edgecolors="black", font_size=8,
          alpha=0.85
          )

  # Labels des arêtes (type ligne ou trafo)
      edge_labels = nx.get_edge_attributes(G, 'type')
      nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)

      plt.title("Réseau électrique test")
      plt.axis("equal")
      plt.show()


  return G, plot_network()

In [ ]:
G, plot_func = create_digraph()

In [ ]:
m = pyo.ConcreteModel()
# Define base values for the per-unit system
s_base = 100  # MVA
v_base_high = 110 # kV (From the create_network function)
first_parent = 4
m.Nodes = pyo.Set(initialize= [b for b in G.nodes])
m.Lines = pyo.Set(initialize= [l for l in G.edges])
m.i = pyo.Set(initialize=[0, 1]) # Initialize m.i with two generic elements
m.j = pyo.Set(initialize= [0, 1])
m.children = pyo.Set(initialize= [9,10,11,12])
m.parents = pyo.Set(initialize = [4])

m.P = pyo.Param(m.Nodes, initialize = {n: - G.nodes[n].get('P_pu') for n in G.nodes}, domain = pyo.Reals, mutable=True) # Make P mutable
m.P_absolute_value = pyo.Param(m.Nodes, initialize = {n: abs(m.P[n]) for n in G.nodes}, domain = pyo.NonNegativeReals) # Make P mutable

m.min_DSO = pyo.Param(m.parents, initialize = {n: min_power_DSO[n-first_parent]  for n in m.parents}, domain = pyo.Reals)
m.max_DSO = pyo.Param(m.parents, initialize = {n: max_power_DSO[n-first_parent]  for n in m.parents}, domain = pyo.Reals)

m.I_min = pyo.Param(m.Lines, initialize = {(i,j): calculate_current_bounds(G.nodes[i]["vn_kv"])[0] for (i,j) in m.Lines}, domain = pyo.Reals)
m.I_max = pyo.Param(m.Lines, initialize = {(i,j): calculate_current_bounds(G.nodes[i]["vn_kv"])[1] for (i,j) in m.Lines}, domain = pyo.Reals)


# Variables principales
m.F = pyo.Var(m.Lines, m.i, m.j, domain=pyo.Reals)     # active power flow through lines
m.I = pyo.Var(m.Lines,  m.i, m.j, domain=pyo.Reals)  # current flowing through lines
m.theta = pyo.Var(m.Nodes,  m.i, m.j, domain=pyo.Reals)  # phase angle of the voltage
m.V = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.NonNegativeReals)  # voltage magnitude at each node
m.E = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.Reals) #net power injection/consumption
m.P_plus = pyo.Var(m.parents, m.i, m.j, domain=pyo.Reals) #power entering the operational graph
m.P_minus = pyo.Var(m.children, m.i, m.j, domain=pyo.Reals) #power leaving the operational graph

m.P_C_set = pyo.Var(m.children, m.i, domain=pyo.Reals) #vertices of the power envelope at each child node

#m.P_C_min = pyo.Var(m.children, m.i, m.j, domain=pyo.Reals)
#m.P_C_max = pyo.Var(m.children, m.i, m.j,  domain=pyo.Reals)

m.V_P = pyo.Param(m.j, initialize = {0: 0.9, 1: 1.1}, domain =pyo.NonNegativeReals)

m.O = pyo.Var(domain= pyo.NonNegativeReals)
m.tot = pyo.Var(domain= pyo.Reals)
m.aux = pyo.Var(m.children, domain=pyo.Reals)
m.diff_DSO = pyo.Var(m.children, domain=pyo.NonNegativeReals)

for u in G.nodes():
  if G.nodes[u].get('P', 0.0) / s_base == 0:
    m.P[u] = 0
  else:
    G.nodes[u]['P_pu'] = G.nodes[u].get('P', 0.0) / s_base
    m.P[u] = - G.nodes[u]['P_pu']

m.info_DSO_param = pyo.Param(m.children, initialize = {n: m.P[n].value for n in m.children}, domain = pyo.Reals)
"""# Convert P parameter to per-unit
for n in G.nodes:
    # Assuming P is in MW, convert to per-unit
    G.nodes[n]['P'] = G.nodes[n].get('P', 0.0) / s_base
    m.P[n] = G.nodes[n]['P']"""

In [ ]:
#Children nodes consumption
def worst_case_children(m, n, vert_pow, vert_volt):
  return m.P_C_set[n, vert_pow] == m.P_minus[n, vert_pow, vert_volt]
m.worst_case = pyo.Constraint(m.children, m.i, m.j, rule=worst_case_children)


# Auxiliary variable for the absolute value of E (already defined as per-unit)
m.z = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.NonNegativeReals)

m.curt = pyo.Var(m.Nodes, m.i, m.j, domain=pyo.Reals)

def curtailment_def_rule(m,n, vert_pow, vert_volt):

  return m.curt[n, vert_pow, vert_volt] == m.P[n] - m.E[n, vert_pow, vert_volt]

m.curt_def = pyo.Constraint(m.Nodes, m.i, m.j, rule=curtailment_def_rule)
# Constraints to define the absolute value (already defined for per-unit E)
def abs_E_pos_rule(m, n, vert_pow, vert_volt):
    return m.z[n, vert_pow, vert_volt] >= m.curt[n, vert_pow, vert_volt]
m.abs_E_pos = pyo.Constraint(m.Nodes, m.i, m.j, rule=abs_E_pos_rule)

def abs_E_neg_rule(m, n, vert_pow, vert_volt):
    return m.z[n, vert_pow, vert_volt] >= -m.curt[n, vert_pow, vert_volt]
m.abs_E_neg = pyo.Constraint(m.Nodes, m.i, m.j, rule=abs_E_neg_rule)

def upper_bound_rule(m, vert_pow, vert_volt):
    # m.z is per-unit, m.O is per-unit
    return sum(m.z[n, vert_pow, vert_volt] for n in m.Nodes) <= m.O

m.upper_bound = pyo.Constraint(m.i, m.j, rule=upper_bound_rule)

# Current magnitude constraint (I_min, I_max are assumed per-unit)
def current_bounds_rule(m, i, j, vert_pow, vert_volt):
    # m.I is per-unit current
    return pyo.inequality(m.I_min[(i,j)], m.I[i,j, vert_pow, vert_volt], m.I_max[(i,j)])
m.CurrentBounds = pyo.Constraint(m.Lines, m.i, m.j, rule=current_bounds_rule)

def phase_constr_rule(m,n, vert_pow, vert_volt):
    return pyo.inequality(theta_min, m.theta[n, vert_pow, vert_volt], theta_max)

m.phaseConstr = pyo.Constraint(m.Nodes, m.i, m.j, rule=phase_constr_rule)

def dc_power_flow_rule(m, i, j, vert_pow, vert_volt):
    if G[i][j].get('b_pu') is None:
      return pyo.Constraint.Skip
    else:
      return m.F[i, j, vert_pow, vert_volt] == m.V_P[vert_volt]**2 *(G[i][j]['b_pu'] * (
        m.theta[i, vert_pow, vert_volt] - m.theta[j, vert_pow, vert_volt])
    )

m.DCFlow = pyo.Constraint(m.Lines, m.i, m.j, rule=dc_power_flow_rule)


def current_def_rule(m, n, i, j, vert_pow, vert_volt):
    # This constraint relates per-unit current, per-unit voltage, and per-unit power flow.
    # In per-unit, P_pu = V_pu * I_pu. This is correct.
    return np.sqrt(3)* m.I[i,j, vert_pow, vert_volt] * m.V_P[vert_volt] == m.F[i,j, vert_pow, vert_volt]

m.current_def = pyo.Constraint(m.Nodes, m.Lines, m.i, m.j, rule=current_def_rule)



"""def nodes_balance(m, n, vert_pow, vert_volt):
    inflow = sum(m.F[k, n, vert_pow, vert_volt] for k in G.predecessors(n) if (k, n) in m.Lines)
    outflow = sum(m.F[n, j, vert_pow, vert_volt] for j in G.successors(n) if (n, j) in m.Lines)
    if n in m.parents:
      return inflow - outflow  ==  m.E[n, vert_pow, vert_volt] - m.P_plus[n, vert_pow, vert_volt]

    if n in m.children:
      return inflow - outflow  ==  m.E[n, vert_pow, vert_volt] + m.P_minus[n, vert_pow, vert_volt]

    else:
      return inflow - outflow ==  m.E[n,vert_pow,vert_volt]

m.nodes_balance = pyo.Constraint(m.Nodes, m.i, m.j, rule=nodes_balance)"""

def power_balance_rule(m, n, vert_pow, vert_volt):
    # Compute net flow into node n by summing over all lines (i,j) in m.Lines
    expr = sum(
        (m.F[i, j, vert_pow, vert_volt] if j == n else 0)
      - (m.F[i, j, vert_pow, vert_volt] if i == n else 0)
      for (i, j) in m.Lines
    )
    # If n is a parent node, subtract P_plus; otherwise use only E[n]
    if n in m.parents:
      return expr == m.E[n, vert_pow, vert_volt] - m.P_plus[n, vert_pow, vert_volt]

    if n in m.children:
      return expr ==  m.E[n, vert_pow, vert_volt] + m.P_minus[n, vert_pow, vert_volt]
    else:
      return expr == m.E[n, vert_pow, vert_volt]

m.power_balance = pyo.Constraint(m.Nodes, m.i, m.j, rule=power_balance_rule)

print(m.min_DSO[4])
def parent_power_constraint_rule(m, parent, vert_pow, vert_volt):
    # m.P_plus is per-unit power entering the operational graph
    return pyo.inequality(min_power_DSO[1], m.P_plus[parent, vert_pow, vert_volt],max_power_DSO[1])

m.parent_power_constraint = pyo.Constraint(m.parents, m.i, m.j, rule=parent_power_constraint_rule)


#Constant voltage assumption
def voltage_constr_rule(m, n, vert_pow, vert_volt):
    return m.V[n, vert_pow, vert_volt] == m.V_P[vert_volt]

m.voltageConstr = pyo.Constraint(m.Nodes, m.i, m.j, rule=voltage_constr_rule)

def children_voltage_rule(m, children, vert_pow, vert_volt):
    return pyo.inequality(V_min, m.V[children, vert_pow, vert_volt], V_max)

m.children_voltage = pyo.Constraint(m.children, m.i, m.j, rule=children_voltage_rule)

"""def dernier_chance_rule(m, child, vert_pow, vert_volt):
    return  m.P_minus[child, vert_pow, vert_volt] >=0
m.dernier_chance = pyo.Constraint(m.children, m.i, m.j, rule=dernier_chance_rule)"""
def aux_constraint_rule(m, n):
  return m.aux[n] == m.P_C_set[n, 0] - m.P_C_set[n, 1]
m.aux_constraint = pyo.Constraint(m.children, rule=aux_constraint_rule)

"""def logical_constraint_rule(m, n):
  return m.P_C_set[n, 0] >= m.P_C_set[n, 1]
m.logical_constraint = pyo.Constraint(m.children, rule=logical_constraint_rule)"""

"""def info_DSO_min_rule(m,n):
  return m.P_C_set[n,1] <= m.info_DSO_param[n]
m.info_DSO_min = pyo.Constraint(m.children, rule=info_DSO_min_rule)

def info_DSO_max_rule(m,n):
  return m.P_C_set[n,0] >= m.info_DSO_param[n]
m.info_DSO_max = pyo.Constraint(m.children, rule=info_DSO_max_rule)"""

def tot_rule(m):
  return m.tot == sum(m.aux[n] for n in m.children)
m.tot_constraint = pyo.Constraint(rule=tot_rule)

def diff_DSO_rule(m,n):
  return - m.diff_DSO[n] <= (m.P_C_set[n,0] + m.P_C_set[n,1])/2 - m.info_DSO_param[n]
m.diff_DSO_constraint = pyo.Constraint(m.children, rule=diff_DSO_rule)

def diff_bis_dso_rule(m,n):
  return (m.P_C_set[n,0] + m.P_C_set[n,1])/2 - m.info_DSO_param[n] <= m.diff_DSO[n]
m.diff_bis_dso_constraint = pyo.Constraint(m.children, rule=diff_bis_dso_rule)


m.abs_diff_DSO = pyo.Var(m.children, domain=pyo.NonNegativeReals)
"""def abs_different_rule(m, n):
    return  m.abs_diff_DSO[n] >= -m.diff_DSO[n]
m.abs_different = pyo.Constraint(m.children, rule=abs_different_rule)

def abs_different_bis_rule(m, n):
    return  m.abs_diff_DSO[n] >= m.diff_DSO[n]
m.abs_different = pyo.Constraint(m.children, rule=abs_different_rule)"""

m.tot_bis = pyo.Var(domain=pyo.Reals)
def tot_bis_rule(m):
  return m.tot_bis == sum(m.diff_DSO[n] for n in m.children)
m.tot_bis_constraint = pyo.Constraint(rule=tot_bis_rule)

# Build sets based on sign of parameter P
m.PositiveNodes = pyo.Set(initialize=[n for n in m.Nodes if pyo.value(m.P[n]) >= 0])
m.NegativeNodes = pyo.Set(initialize=[n for n in m.Nodes if pyo.value(m.P[n]) <= 0])

# Constraint for positive P: upper bound
def net_power_upper_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] <= m.P[n]
m.net_power_upper = pyo.Constraint(m.PositiveNodes, m.i, m.j, rule=net_power_upper_rule)

def sign_E_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] >= 0

m.sign_E = pyo.Constraint(m.PositiveNodes, m.i, m.j, rule=sign_E_rule)



# Constraint for negative P: lower bound
def net_power_lower_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] >= m.P[n]
m.net_power_lower = pyo.Constraint(m.NegativeNodes, m.i, m.j, rule=net_power_lower_rule)

def sign_E_lower_rule(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] <= 0

m.sign_E_lower = pyo.Constraint(m.NegativeNodes, m.i, m.j, rule=sign_E_lower_rule)

"""m.PositiveNodes = pyo.Set(initialize=[n for n in m.Nodes if m.P[n] >= 0])
m.NegativeNodes = pyo.Set(initialize=[n for n in m.Nodes if m.P[n] <= 0])

# Constraint for positive P: upper bound
def net_power_upper(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] <= m.P[n]
m.net_power_upper = pyo.Constraint(m.PositiveNodes, m.i, m.j, rule=net_power_upper)

# Constraint for negative P: lower bound
def net_power_lower(m, n, vert_pow, vert_volt):
    return m.E[n, vert_pow, vert_volt] >= m.P[n]
m.net_power_lower = pyo.Constraint(m.NegativeNodes, m.i, m.j, rule=net_power_lower)"""

"""# Auxiliary variables for the L1 norm of P_C_set
m.abs_P_C_set = pyo.Var(m.children, domain=pyo.NonNegativeReals)

# Constraints to linearize the L1 norm of P_C_set
def abs_P_C_set_pos_rule(m, n):
    return m.abs_P_C_set[n] >= m.aux[n]
m.abs_P_C_set_pos = pyo.Constraint(m.children, rule=abs_P_C_set_pos_rule)

def abs_P_C_set_neg_rule(m, n):
    return m.abs_P_C_set[n] >= -m.aux[n]
m.abs_P_C_set_neg = pyo.Constraint(m.children, rule=abs_P_C_set_neg_rule)"""

In [ ]:
beta = 3
alpha = 1
enveloppe_taille = []
curtail = []
close = []


print(alpha)
def objective_rule(m):
  return  m.tot - alpha*m.O - beta * m.tot_bis
m.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)
params = {
    "WLSACCESSID": 'efa10779-150a-495c-ab62-6a20ff44c598',
    "WLSSECRET": 'fa980d94-a6fd-4d7b-aa68-b0e19ddd2e84',
    "LICENSEID": 2651433,
           }
env = gp.Env(params=params)
solver = pyo.SolverFactory('gurobi', env= env)
results = solver.solve(m, tee=True)

  # Check the termination condition before accessing results
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
  curtail.append(m.O.value)
  enveloppe_taille.append(m.tot.value)
  close.append(m.tot_bis.value)

else:

      # Append None for non-optimal results to maintain list length
  curtail.append(None)
  enveloppe_taille.append(None)
  close.append(None)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract data for plotting
children_nodes = list(m.children)
p_c_set_0_values = [m.P_C_set[n, 0].value for n in m.children]
p_c_set_1_values = [m.P_C_set[n, 1].value for n in m.children]
info_dso_values = [m.info_DSO_param[n] for n in m.children]

# Create the plot
plt.figure(figsize=(5, 6))

# Adjust x_pos to control spacing (e.g., reduce the multiplier)
x_pos = np.arange(len(children_nodes)) * 0.0005 # Reduced spacing by multiplying by 0.0005

# Plot lines between P_C_set[n][0] and P_C_set[n][1] for each child node
for i in range(len(children_nodes)):
    plt.plot([x_pos[i], x_pos[i]], [p_c_set_1_values[i], p_c_set_0_values[i]], marker='o', linestyle='-', color='blue')

# Plot only dots for info_DSO_param
plt.plot(x_pos, info_dso_values, marker='s', linestyle='', label=f'DSO power demand estimation')

# Create legend handles for the P_C_set endpoints
# We plot invisible points with the desired labels to create legend entries
plt.plot([], [], marker='o', linestyle='-', color='blue', label=f'Power envelope ($\\alpha$={alpha}, $\\beta$={beta})')


plt.xticks(x_pos, children_nodes)
plt.xlabel('Child Node Index')
plt.ylabel('Power (in per-unit)')
#plt.title('Comparison of P_C_set values and DSO estimation for Child Nodes')
plt.legend(loc='upper left') # Move the legend to the upper left
plt.grid(True)

# Draw labels to the left of the points
# for i, n in enumerate(children_nodes):
#     # Label for the upper bound of the envelope
#     plt.text(x_pos[i], p_c_set_0_values[i], f"{round(p_c_set_0_values[i], 4)}", fontsize=8, ha='center', va='bottom', color='red')
#     # Label for the lower bound of the envelope
#     plt.text(x_pos[i], p_c_set_1_values[i], f"{round(p_c_set_1_values[i], 4)}", fontsize=8, ha='center', va='top', color='red')
#     # Label for the DSO estimation
#     plt.text(x_pos[i], info_dso_values[i], f"{round(info_dso_values[i], 4)}", fontsize=8, ha='center', va='center', color='green')

# Ensure the y-axis is visible
plt.autoscale(enable=True, axis='y', tight=False)

# Adjust plot layout to prevent labels from being cut off
plt.tight_layout()

# Save the plot as a PDF file
plt.savefig('child_nodes_envelopes.pdf')

plt.show()

In [ ]:
# Print values of m.I
print("Values of m.I:")
for line in m.Lines:
    u, v = line
    for vert_pow in m.i:
        for vert_volt in m.j:
            try:
                print(f"m.I[{u},{v},{vert_pow},{vert_volt}].value: {m.I[u, v, vert_pow, vert_volt].value}")
            except:
                print(f"m.I[{u},{v},{vert_pow},{vert_volt}].value: None")


# Print values of m.I_max
print("\nValues of m.I_max:")
for (u, v) in m.Lines:
  print(m.I_max[(u,v)])